In [1]:
from utilities import *
from pytorch_nn import Lipreading1
from pytorch_blocks import _3d_block, ConvNet1D, threeD_to_2D_tensor, noop, _average_batch, _conv_block, _bottleneck_conv_block
from lips_points_dataloading import dataloader_for_lips
from Model_training_blocks import FullTrainer
from fastai.layers import *
import torch
from torch import nn
import torch.nn.functional as F
from torchvision import models

from with_other_resnet import TrueResNet, BasicBlock

In [2]:
words_list_file = '/home/taylorpap/Bootcamp/wordlist.txt'
words_list = get_wordslist_from_txt_file(words_list_file)

In [3]:
temp_words_list = []
for index in range(1, len(words_list), 10):
    temp_words_list.append(words_list[index])
print(len(temp_words_list))

50


In [4]:
class LipreadLSTMv1(nn.Module):
    def __init__(self, num_classes, pretrained_res=False, relu_type='prelu'):
        super(LipreadLSTMv1, self).__init__()
        self.num_classes = num_classes
        self.frontend_out = 64
        self.backend_out = 512

        self.front3D = _3d_block(1, self.frontend_out, kernel_size=(5, 7, 7), stride=(1, 2, 2), padding=(2, 3, 3), bias=False)
        self.max_pool_3D = nn.MaxPool3d(kernel_size=(1, 3, 3), stride=(1, 2, 2), padding=(0, 1, 1))
        self.resnet = TrueResNet(BasicBlock, [2, 2, 2, 2], relu_type=relu_type)
        self.lstm = nn.LSTM(
            input_size=self.backend_out,
            hidden_size=256,
            num_layers=3,
            batch_first=True,
            bidirectional=True,
            dropout=0.1
        )
        self.conv1 = nn.Conv1d(512, 256, kernel_size=3)
        self.batchnorm1 = nn.BatchNorm1d(256)
        if relu_type == 'relu':
            self.relu1 = nn.ReLU()
        elif relu_type == 'prelu':
            self.relu1 = nn.PReLU(num_parameters=256)
        self.fc2 = nn.Linear(256, num_classes)

        if pretrained_res:
            pretrained_model = models.resnet18(pretrained=True)
            #pretrained_model.fc = nn.Linear(pretrained_model.fc.in_features, num_classes)
            self.resnet.load_state_dict(pretrained_model.state_dict(), strict=False)
            for param in self.resnet.parameters():
                param.requires_grad = False


    def forward(self, x, lengths):
        B, C, T, H, W = x.size()
        #print("Initial Shape: " + str(x.shape))
        x = self.front3D(x)
        #print("3D Out Shape: " + str(x.shape))
        x = self.max_pool_3D(x)
        #print("3D Max Pool Shape: " + str(x.shape))
        Tnew = x.shape[2]
        #transposed = x.transpose(1, 2).contiguous()
        #x = transposed.view(-1, 64, x.size(3), x.size(4))
        x = threeD_to_2D_tensor(x)
        #print("ResNet In Shape: " + str(x.shape))
        x = self.resnet(x)
        #print("ResNet Out Shape: " + str(x.shape))
        x = x.view(B, Tnew, x.size(1))
        #x = x.transpose(1, 2)
        #print("DataTransform Out Shape: " + str(x.shape))

        #x = self.fc(x)
        #x = self.bn2(x)
        #print("First Batch Norm OutShape: " + str(x.shape))
        #x = x.view(x.shape[0], -1, self.num_classes)
        x, _ = self.lstm(x)
        #print("LSTM Out Shape: " + str(x.shape))
        x = x.transpose(1, 2)
        #print("Transposed Shape: {}".format(str(x.shape)))
        x = self.conv1(x)
        x = self.batchnorm1(x)
        x = self.relu1(x)
        #print("Avg Batch InShape: " + str(x.shape))
        x = _average_batch(x, lengths)
        #print("Avg Batch OutShape: " + str(x.shape))
        x = self.fc2(x)
        #print("Last Liner Out Shape: " + str(x.shape))
        return x

In [4]:
data_path = '/media/taylorpap/2TBM2/Dataset_2/GreyCropped'
from datasetloadingpy import dataloaders
datasets = dataloaders(data_dir=data_path, label_fp=temp_words_list, batch_size=32, workers=8)

Partition train loaded
Partition test loaded
Partition val loaded


In [5]:
train_features, train_lengths, train_labels = next(iter(datasets['train']))
print(f"Feature batch shape: {train_features.size()}")
print(f"Labels batch shape: {train_labels.size()}")

Feature batch shape: torch.Size([32, 29, 88, 88])
Labels batch shape: torch.Size([32])


TESTING LSTM MODEL WITH VARIOUS LOSS FUNCTIONS:

In [7]:
model1 = LipreadLSTMv1(len(temp_words_list), pretrained_res=True) #Using Loaded Model Weights for ResNet18

/home/taylorpap/anaconda3/envs/projectenv/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/taylorpap/anaconda3/envs/projectenv/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [8]:
if torch.cuda.is_available():
    model1.cuda()

In [9]:
save_direc = '/home/taylorpap/Bootcamp/test_model1_CrossEntLoss/'
train_model = FullTrainer(model1, datasets, criterion=nn.CrossEntropyLoss(), epochs=40, save_dir=save_direc)

Current Epoch: 0
Running Loss: 187487.15158700943, Running Corrects: 2073.0, Running All: 48623.0


100%|██████████| 79/79 [00:04<00:00, 18.69it/s]

2499 in total	CR: 0.060024009603841535
val Epoch:	 0	Loss val: 3.7480	Acc val:0.0600, LR: 0.003
Epoch len: 189.77115988731384 Estimated Remaining: 123.351253926754 Min
Current Epoch: 1


Running Loss: 178847.27205729485, Running Corrects: 3572.0, Running All: 48623.0


100%|██████████| 79/79 [00:05<00:00, 13.93it/s]


...

2499 in total	CR: 0.8403361344537815
val Epoch:	38	Loss val: 0.5331	Acc val:0.8403, LR: 4.144511940348517e-05
Epoch len: 156.28018522262573 Estimated Remaining: 2.604669753710429 Min
Current Epoch: 39
Running Loss: 17063.89448094368, Running Corrects: 43269.0, Running All: 48623.0


100%|██████████| 79/79 [00:03<00:00, 21.34it/s]


2499 in total	CR: 0.8415366146458584
val Epoch:	39	Loss val: 0.5336	Acc val:0.8415, LR: 1.846748910729351e-05
Epoch len: 156.5054190158844 Estimated Remaining: 0.0 Min


 77%|███████▋  | 61/79 [00:02<00:00, 24.31it/s]/home/taylorpap/PycharmProjects/Capstone/datasetloadingpy.py:229: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:230.)
  data = torch.FloatTensor(data)
100%|██████████| 79/79 [00:03<00:00, 21.07it/s]

2497 in total	CR: 0.8458149779735683
Test time performance of best epoch: 0.8458149779735683 (loss: 0.5099420872124186)


In [31]:
#Clear the Notebook/GPU cache to try training another model
model1 = None
torch.cuda.empty_cache()

In [34]:
model2 = LipreadLSTMv1(len(temp_words_list), pretrained_res=True) #Using Loaded Model Weights for ResNet18

In [35]:
if torch.cuda.is_available():
    model2.cuda()

In [36]:
save_direc = '/home/taylorpap/Bootcamp/test_model2_BCE/'
train_model = FullTrainer(model2, datasets, criterion=nn.NLLLoss(), epochs=40, save_dir=save_direc)

Current Epoch: 0
Running Loss: nan, Running Corrects: 1111.0, Running All: 48623.0


100%|██████████| 79/79 [00:03<00:00, 20.21it/s]


2499 in total	CR: 0.020008003201280513
val Epoch:	 0	Loss val: nan	Acc val:0.0200, LR: 0.003
Epoch len: 166.21943378448486 Estimated Remaining: 108.04263195991516 Min
Current Epoch: 1


KeyboardInterrupt: 

In [27]:
#Clear the Notebook/GPU cache to try training another model
model2 = None
torch.cuda.empty_cache()

In [15]:
model3 = LipreadLSTMv1(len(temp_words_list), pretrained_res=True) #Using Loaded Model Weights for ResNet18
if torch.cuda.is_available():
    model3.cuda()

/home/taylorpap/anaconda3/envs/projectenv/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/taylorpap/anaconda3/envs/projectenv/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [16]:
save_direc = '/home/taylorpap/Bootcamp/test_model3_L1/'
train_model = FullTrainer(model3, datasets, criterion=nn.L1Loss(), epochs=40, save_dir=save_direc)

Current Epoch: 0


/home/taylorpap/anaconda3/envs/projectenv/lib/python3.10/site-packages/torch/nn/modules/loss.py:101: UserWarning: Using a target size (torch.Size([32])) that is different to the input size (torch.Size([32, 50])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.l1_loss(input, target, reduction=self.reduction)


RuntimeError: The size of tensor a (50) must match the size of tensor b (32) at non-singleton dimension 1

In [32]:
model3 = None
torch.cuda.empty_cache()

In [18]:
model4 = Lipreading1(len(temp_words_list))
if torch.cuda.is_available():
    model4.cuda()

In [20]:
save_direc = '/home/taylorpap/Bootcamp/test_model4_CrossEntLoss/'
train_model = FullTrainer(model4, datasets, criterion=nn.CrossEntropyLoss(), epochs=40, save_dir=save_direc)

Current Epoch: 0
Running Loss: 191489.91651058197, Running Corrects: 1589.0, Running All: 48623.0


100%|██████████| 79/79 [00:03<00:00, 20.95it/s]


2499 in total	CR: 0.05002000800320128
val Epoch:	 0	Loss val: 3.8313	Acc val:0.0500, LR: 0.003
Epoch len: 290.7344219684601 Estimated Remaining: 188.97737427949906 Min
Current Epoch: 1
Running Loss: 180191.0909423828, Running Corrects: 3197.0, Running All: 48623.0


100%|██████████| 79/79 [00:03<00:00, 20.87it/s]


2499 in total	CR: 0.09843937575030012
val Epoch:	 1	Loss val: 3.5684	Acc val:0.0984, LR: 0.003
Epoch len: 291.9331479072571 Estimated Remaining: 184.89099367459616 Min
Current Epoch: 2
Running Loss: 156093.2252459526, Running Corrects: 8208.0, Running All: 48623.0


...

2499 in total	CR: 0.8723489395758304
val Epoch:	38	Loss val: 0.4377	Acc val:0.8723, LR: 4.144511940348517e-05
Epoch len: 277.0460395812988 Estimated Remaining: 4.617433993021647 Min
Current Epoch: 39
Running Loss: 4476.804649874568, Running Corrects: 47538.0, Running All: 48623.0


100%|██████████| 79/79 [00:03<00:00, 20.24it/s]


2499 in total	CR: 0.8735494197679071
val Epoch:	39	Loss val: 0.4365	Acc val:0.8735, LR: 1.846748910729351e-05
Epoch len: 277.0163724422455 Estimated Remaining: 0.0 Min


 78%|███████▊  | 62/79 [00:03<00:00, 22.94it/s]/home/taylorpap/PycharmProjects/Capstone/datasetloadingpy.py:229: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:230.)
  data = torch.FloatTensor(data)
100%|██████████| 79/79 [00:03<00:00, 20.02it/s]

2497 in total	CR: 0.8706447737284742
Test time performance of best epoch: 0.8706447737284742 (loss: 0.4399932668860263)


In [33]:
model4 = None
torch.cuda.empty_cache()

In [6]:
def _2d_stem(*sizes):
    return nn.Sequential(*[
        ConvLayer(sizes[i], sizes[i+1], 3, stride=2 if i==0 else 1)
            for i in range(len(sizes)-1)
    ] + [nn.MaxPool2d(kernel_size=(3,3), stride=(2,2))])

In [11]:
class ResBlock(nn.Module):
    def __init__(self, ni, nf, stride=1, bottled=False):
        super(ResBlock, self).__init__()
        self.convs = _bottleneck_conv_block(ni, nf, stride) if bottled else _conv_block(ni, nf, stride)
        self.idconv = noop if ni == nf else ConvLayer(ni, nf, 1, act_cls=None)
        self.pool = noop if stride == 1 else nn.AvgPool2d(2, ceil_mode=True)

    def forward(self, x):
        return F.relu(self.convs(x) + self.idconv(self.pool(x)))


class ResNet(nn.Sequential):
    def __init__(self, layers, expansion=1):

        #self.relu_type= relu_type
        self.bottled = True if expansion > 1 else False
        self.block_sizes = [64, 128, 256]
        for i in range(1, len(self.block_sizes)): self.block_sizes[i] *= expansion
        blocks = [self._make_layer(*o) for o in enumerate(layers)]

        super().__init__(*blocks, nn.AdaptiveAvgPool2d(1))

    def _make_layer(self, idx, n_layers):
        stride = 1 if idx==0 else 2
        ch_in, ch_out = self.block_sizes[idx:idx+2]
        return nn.Sequential(*[
            ResBlock(ch_in if i==0 else ch_out, ch_out, stride if i==0 else 1, self.bottled)
            for i in range(n_layers)
        ])

class Chomp1d(nn.Module):
    def __init__(self, chomp_size, symm_chomp):
        super(Chomp1d, self).__init__()
        self.chomp_size = chomp_size
        self.symm_chomp = symm_chomp
        if self.symm_chomp:
            assert self.chomp_size % 2 == 0, "If symmetric chomp, chomp size needs to be even"
    def forward(self, x):
        if self.chomp_size == 0:
            return x
        if self.symm_chomp:
            return x[:, :, self.chomp_size//2:-self.chomp_size//2].contiguous()
        else:
            return x[:, :, :-self.chomp_size].contiguous()

In [12]:
class BasicBlock1D(nn.Module):
    def __init__(self, n_inputs, n_outputs, kernel_size, stride, dilation, padding, dropout=0.2, relu_type='relu'):
        super(BasicBlock1D, self).__init__()

        self.conv1 = nn.Conv1d(n_inputs, n_outputs, kernel_size, stride=stride, padding=padding, dilation=dilation)
        self.batchnorm1 = nn.BatchNorm1d(n_outputs)
        self.chomp1 = Chomp1d(padding, True)
        if relu_type == 'relu':
            self.relu1 = nn.ReLU()
        elif relu_type == 'prelu':
            self.relu1 = nn.PReLU(num_parameters=n_outputs)
        self.dropout1 = nn.Dropout(dropout)
        self.conv2 = nn.Conv1d(n_outputs, n_outputs, kernel_size, stride=stride, padding=padding, dilation=dilation)
        self.batchnorm2 = nn.BatchNorm1d(n_outputs)
        self.chomp2 = Chomp1d(padding, True)
        if relu_type == 'relu':
            self.relu2 = nn.ReLU()
        elif relu_type == 'prelu':
            self.relu2 = nn.PReLU(num_parameters=n_outputs)
        self.dropout2 = nn.Dropout(dropout)

        self.downsample = nn.Conv1d(n_inputs, n_outputs, 1) if n_inputs != n_outputs else None
        if relu_type == 'relu':
            self.relu = nn.ReLU()
        elif relu_type == 'prelu':
            self.relu = nn.PReLU(num_parameters=n_outputs)

    def forward(self, x):
        out = self.conv1(x)
        out = self.batchnorm1(out)
        out = self.chomp1(out)
        out = self.relu1(out)
        out = self.dropout1(out)

        out = self.conv2(out)
        out = self.batchnorm2(out)
        out = self.chomp2(out)
        out = self.relu2(out)
        out = self.dropout2(out)

        res = x if self.downsample is None else self.downsample(x)
        return self.relu(out + res)


class ConvNet1D(nn.Module):
    def __init__(self, num_inputs, num_channels, kernel_size=3, dropout=0.2, relu_type='relu'):
        super(ConvNet1D, self).__init__()

        layers = []
        for i in range(len(num_channels)):
            dilation_size = 2 ** i
            in_channels = num_inputs if i==0 else num_channels[i-1]
            out_channels = num_channels[i]
            layers.append(BasicBlock1D(in_channels, out_channels, kernel_size, stride=1, dilation=dilation_size,
                                       padding=(kernel_size-1) * dilation_size, dropout=dropout, relu_type=relu_type))
        self.network = nn.Sequential(*layers)

    def forward(self, x, lengths):
        x = self.network(x)
        x = _average_batch(x, lengths)
        return x

In [15]:
class Lipread2(nn.Module):
    def __init__(self, num_classes, relu_type = 'prelu'):
        super(Lipread2, self).__init__()
        self.kernel_size = 3
        self.dropout = 0.2
        self.frontend_out = 64
        self.backend_out = 256
        self.expansion = 4
        self.convolution_channels = [512, 256, 128]
        self.num_classes = num_classes

        self.frontend3D = _3d_block(1, 32, kernel_size=(5,7,7), stride=(1,2,2), padding=(2,3,3))
        self.max_pool1 = nn.MaxPool3d(kernel_size=(1,3,3), stride=(1,2,2), padding=(0,1,1))
        self.stem = _2d_stem(32, 32, 64)
        self.trunk = ResNet([3,3], expansion=self.expansion)

        self.tcn = ConvNet1D(self.backend_out * self.expansion, self.convolution_channels, kernel_size=self.kernel_size, dropout=self.dropout, relu_type=relu_type)
        self.convnet_output = nn.Linear(self.convolution_channels[-1], num_classes)

    def forward(self, x, lengths):
        B, C, T, H, W = x.size()
        x = self.frontend3D(x)
        x = self.max_pool1(x)
        Tnew = x.shape[2]
        x = threeD_to_2D_tensor(x)
        x = self.stem(x)
        x = self.trunk(x)
        x = x.view(x.size(0), -1)
        x = x.view(B, Tnew, x.size(1))
        x = x.transpose(1, 2)
        x = self.tcn(x, lengths)
        return self.convnet_output(x)


In [16]:
model5 = Lipread2(len(temp_words_list))
if torch.cuda.is_available():
    model5.cuda()

In [17]:
save_direc = '/home/taylorpap/Bootcamp/test_model5/'
train_model = FullTrainer(model5, datasets, criterion=nn.CrossEntropyLoss(), epochs=40, save_dir=save_direc)

Current Epoch: 0
Running Loss: 190583.4138572216, Running Corrects: 1891.0, Running All: 48623.0


100%|██████████| 79/79 [00:02<00:00, 27.82it/s]

2499 in total	CR: 0.056022408963585436
val Epoch:	 0	Loss val: 3.7851	Acc val:0.0560, LR: 0.003
Epoch len: 121.54991817474365 Estimated Remaining: 79.00744681358337 Min
Current Epoch: 1


Running Loss: 164828.2197751999, Running Corrects: 6609.0, Running All: 48623.0


100%|██████████| 79/79 [00:02<00:00, 28.88it/s]


...

2499 in total	CR: 0.891156462585034
val Epoch:	38	Loss val: 0.3974	Acc val:0.8912, LR: 4.144511940348517e-05
Epoch len: 116.48789477348328 Estimated Remaining: 1.941464912891388 Min
Current Epoch: 39
Running Loss: 1773.4379055444151, Running Corrects: 48255.0, Running All: 48623.0


100%|██████████| 79/79 [00:02<00:00, 30.20it/s]


2499 in total	CR: 0.8927571028411364
val Epoch:	39	Loss val: 0.3968	Acc val:0.8928, LR: 1.846748910729351e-05
Epoch len: 119.11651062965393 Estimated Remaining: 0.0 Min


 78%|███████▊  | 62/79 [00:02<00:00, 33.80it/s]/home/taylorpap/PycharmProjects/Capstone/datasetloadingpy.py:229: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:230.)
  data = torch.FloatTensor(data)
100%|██████████| 79/79 [00:03<00:00, 26.26it/s]

2497 in total	CR: 0.880656788145775
Test time performance of best epoch: 0.880656788145775 (loss: 0.4262918140396064)
